In [1]:
import ee


In [2]:
ee.Initialize()

In [3]:
from cnwi.fourier_transform import compute_fourier_transform

In [4]:
# cloud mask function
def mask_l8_sr(image: ee.Image):
    """Masks clouds in Landsat 8 SR image."""
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(2 ** 4).eq(0)
    saturation_mask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(optical_bands, None, True) \
        .addBands(thermal_bands, None, True) \
        .updateMask(qa_mask) \
        .updateMask(saturation_mask)


In [5]:
from typing import Callable
def add_ndvi(nir: str, red: str) -> Callable[[ee.Image], ee.Image]:
    """Adds NDVI band to an image."""
    def _add_ndvi(image: ee.Image) -> ee.Image:
        return image.addBands(image.normalizedDifference([nir, red]).rename('NDVI'))
    return _add_ndvi

In [6]:
# Image Collection Pre Processing
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2018', '2022')
    .filterBounds(ee.Geometry.Point(-77.17593890457817, 44.095224852158104))
    .map(mask_l8_sr)
    .map(add_ndvi('SR_B5', 'SR_B4'))
)

In [7]:
# Compute Fourier Transform from the pre processed image collection
fourier_transform = compute_fourier_transform(collection, 'NDVI')

In [8]:
fourier_transform.bandNames().getInfo()

['NDVI',
 'constant_coef',
 't_coef',
 'cos_1_coef',
 'cos_2_coef',
 'cos_3_coef',
 'sin_1_coef',
 'sin_2_coef',
 'sin_3_coef',
 'phase_1',
 'amp_1',
 'phase_2',
 'amp_2',
 'phase_3',
 'amp_3']

In [9]:
# visualize the results
phase = fourier_transform.select('phase_1')
amplitude = fourier_transform.select('amp_1').multiply(5) # multiply by 5 to make it more visible
dependent = fourier_transform.select('NDVI')

composite = ee.Image([phase, amplitude, dependent]).hsvToRgb()


In [11]:
# Visualize the results
import geemap 
Map = geemap.Map()
Map.addLayer(composite, {}, 'composite')

Map.addLayerControl()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…